# ECON526: Assignment 5

Jesse Perla, Paul Schrimpf, and Phil Solimine

## Setup

Feel free to use the following packages (and we have added a few
convenience imports)

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy
from scipy import stats
import pandas as pd

# Q1

## Q1.1

The file `practice_data.csv`, inside the `data` folder, contains data
from a simulated experiment.

Read the dataset into a pandas dataframe. The data contains 2 columns,
`treatment` and `outcome`. The `treatment` column is a binary variable
that indicates whether the subject was assigned to the treatment group
(1) or the control group (0). The `outcome` column is a continuous
variable that indicates the outcome of the experiment for each subject.

Print the head of this dataframe.

In [2]:
# your code here

Create a histogram of the outcome variable, which has the distributions
of the outcome for the treatment and control groups overlaid on top of
each other in different colors. Make the histogram transparent so that
you can clearly see both distributions. Add a legend to the plot to
indicate which distribution is which.

In [3]:
# your code here

## Q1.2

Estimate the average treatment effect (ATE) using the difference in
sample means as your estimator. Print each sample average and your point
estimate of the ATE as formatted strings. Interpret your result in the
markdown cell below.

In [7]:
# your code here

(double click to edit your answer)

## Q1.3

Now use the standard error formula to calculate the asymptotic standard
errors of each of the sample means, and use these to construct a 95%
confidence interval for the ATE. Print all output in formatted strings.
In the markdown cell below, interpret your confidence interval.

In [9]:
# your code here

(double click to edit your answer)

## Q1.4

Estimate the standard errors again using bootstrapping. You should use
1000 bootstrap samples. Print the standard errors and the 95% confidence
interval for the ATE. How are the results different? Is this what you
expected? Explain in the markdown cell below.

Hint: when you resample, make sure to keep the same proportion of
treatment and control observations as in the original sample. An easier
way to do this might be to split the data into treatment and control
groups, resample each group separately, and then combine the resampled
groups back into a single dataframe.

In [11]:
np.random.seed(100) # do not change this line
# your code here

(double click to edit your answer)

# Q2

## Q2.1

In class we analyzed the effect of online class formats on student
scores in a course. We found that the online format produced a negative
effect on scores.

The dataset is stored in `data/online_classroom.csv`. Read the data into
a pandas dataframe and print the head.

In [13]:
# your code here

Create a similar plot to the one we created in class, with the
distribution of scores for each class format (online, face-to-face, and
blended) overlaid on top of each other. Make the plot transparent so
that you can clearly see all three distributions. Add a legend to the
plot to indicate which distribution is which. Draw vertical lines on the
plot (using the `vlines` function) to indicate the mean score for each
class format. Make sure that the distributions and vertical lines are
different colors so that you can tell them apart.

In [14]:
# your code here

## Q2.2

Follow the steps from class to estimate the average treatment effect
(ATE) of the online format, using the blended format as the control
group. Use using the **difference in sample means** as your estimator.
Print each sample average and your point estimate of the ATE, along with
the standard error of each mean and the standard error of the ATE. Can
you reasonably conclude (at the **90% confidence level**) that the
online format causes lower test scores than the blended format? Explain
in the markdown cell below.

In [17]:
# your code here

(double click to edit your answer)

## Q2.3

In class we noticed that there are not all that many observations in the
dataset. It also looks like the distribution of scores is not normal (in
fact, we know it’s not, since the score is bounded below at 0 and above
at 100). So we might be concerned that this, plus the limited amount of
data, means that our asymptotic standard errors are not very accurate.

Use bootstrapping to estimate the standard error of the ATE. You should
use 1000 bootstrap samples. Print the standard errors and the 90%
confidence interval for the ATE. How are the results different?

Is this what you expected? Does it seem like we have enough data for the
estimator to approach its asymptotic distribution? Explain in the
markdown cell below.

Hint: like before, start by splitting the data into treatment (online)
and control (blended) groups. Define a function that will resample the
dataframes. Then resample each group separately to maintain the right
proportion of treatment and control observations.

In [19]:
np.random.seed(100) # do not change this line
# your code here

(double click to edit your answer)